In [1]:
import pandas as pd

In [2]:
matches = pd.read_csv('matches.csv')

In [6]:
matches.shape
matches['Team'].value_counts()

Team
Liverpool                   114
Southampton                 114
Watford                     114
Bournemouth                 114
West Ham United             114
Brighton and Hove Albion    114
Manchester City             114
Newcastle United            114
Everton                     114
Crystal Palace              114
Burnley                     114
Arsenal                     114
Tottenham Hotspur           114
Leicester City              114
Chelsea                     114
Manchester United           114
Wolverhampton Wanderers      76
Huddersfield Town            76
Sheffield United             38
Aston Villa                  38
Norwich City                 38
Cardiff City                 38
Fulham                       38
Swansea City                 38
Stoke City                   38
West Bromwich Albion         38
Name: count, dtype: int64